In [ ]:
"""

Prepare the masks of the ice shelves based on BedMachine

"""

In [ ]:
import xarray as xr
import numpy as np
from pyproj import Transformer
import pandas as pd
from tqdm.notebook import trange, tqdm
import cc3d


import multimelt.plume_functions as pf
import multimelt.box_functions as bf
import multimelt.useful_functions as uf
import basal_melt_param.create_isf_mask_functions as isfmf

import distributed

In [ ]:
%matplotlib qt5

In [ ]:
map_lim = [-3000000,3000000]

#chunk_size = 700
chunk_size = False

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask ='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
outputpath_boxes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'
outputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'

file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_isf_conc = file_mask_orig_cut['isf_conc']

xx = file_msk['x']
yy = file_msk['y']

dx = abs(xx[1] - xx[0])
dy = abs(yy[1] - yy[0])

In [ ]:
        # do some fine-tuning for overlapping ice shelves   
        problem_regions = [2,3,8,9,10,13,23,26,27,28,29,32,34,38,44,46,50,57,59,60,
                   63,70,71,72,73,74,76,77,78,83,84,85,89,91,96,103]
        
        for conn_label in range(1,labels_out.max()):
            basins_conn_domain = arr_def_ismask['ID_isf'].where(labelled_isf == conn_label, drop=True)
            max_label = basins_conn_domain.max().values
            min_label = basins_conn_domain.min().values
            
            # for areas with two labels in problem regions, take the one with the most points
            if max_label != min_label:
                groups_isf = basins_conn_domain.groupby(basins_conn_domain)
                groups_labels = groups_isf.groups.keys()
                if groups_isf.count().ID_isf.count() > 1:
                    if any(x in problem_regions for x in list(groups_labels)):
                        #print(conn_label)
                        #print(min_label,max_label)
                        dominant_isf = groups_isf.count().idxmax().values
                        if dominant_isf == 12:
                            dominant_isf = 14
                        #print(dominant_isf)
                        new_mask = new_mask.where(labelled_isf != conn_label, dominant_isf)
            
        # other fine-tuning: if an ice shelf is split, keep the largest connected domain
        dx = abs(file_conc.x[1] - file_conc.x[0])
        dy = abs(file_conc.y[1] - file_conc.y[0])

        split_regions = [70,77,83,89,103] 

        for rreg in split_regions:
            # look where there are the same labels in several unconnected domains
            labels_same = list(new_mask.groupby(labelled_isf).groups) * (new_mask.groupby(labelled_isf).median() == rreg)
            labels_same = labels_same[labels_same>0]

            area_before = 0
            for conn_label in labels_same:
                # compute the area of the different unconnected areas
                conc_for_area = file_conc.where(labelled_isf == conn_label, drop=True)
                area_now = (conc_for_area * dx * dy).sum()
                if area_now >= area_before:
                    area_before = area_now
                    largest_label = conn_label

            # set the smaller areas to 159
            for small_label in (labels_same.where(labels_same != largest_label).dropna('labels')):
                new_mask = new_mask.where(labelled_isf != small_label, 159)

        new_mask = new_mask + 1
        new_mask_info = arr_def_ismask.copy()
        new_mask_info['Nisf'] = new_mask_info['Nisf'] + 1
        
        new_mask = new_mask.where(file_msk != 0, 1).where(file_msk != 2, 0)

In [ ]:
def def_isf_mask(arr_def_ismask, file_msk, file_conc, lon, lat, FRIS_one=True, 
                 IMBIE_mask=True, variable_geometry=False, connectivity = 4, threshold = 4):
    
    """
    Define a mask for the individual ice shelves. 
    
    This function defines a mask for the individual ice shelves. I think it works for both stereographic and latlon grids but I have not tried the latter.
    
    Parameters
    ----------
    arr_def_ismask : np.array
        Array containing minlon,maxlon,minlat,maxlat,is_nb or xr.Dataset with drainage basins
    file_msk : xr.DataArray
        Mask separating ocean (0), ice shelves (between 0 and 2, excluding 0 and 2), grounded ice (2) 
    file_conc : xr.DataArray
        Ice shelf concentration for each point (between 0 and 1)
    lon : xr.DataArray
        Longitude (depends on x,y for stereographic)
    lat : xr.DataArray
        Latitude (depends on x,y for stereographic)
    FRIS_one : Boolean 
        If True, Filchner-Ronne are considered as one ice-shelf
    mouginot_basins : Boolean 
        If True, arr_def_ismask is an xr.DataArray with drainage basins
    variable_geometry : Boolean 
        If True, arr_def_ismask
    connectivity : int
        4 or 8 for 2D, defines what is considered a "connected" point
    threshold : int
        Size of lonely pixel areas to remove
        
    Returns
    -------
    new_mask : xr.DataArray
        Array showing the coverage of each ice shelf with the respective ID, open ocean is 1, land is 0
    """    
    
    if IMBIE_mask:
        
        isf_mask = file_msk.copy()
        # only ice shelves
        isf_only_mask = file_conc > 0
        
        #find connected components
        dusted = cc3d.dust(isf_only_mask.values.astype(np.int64), 
                   threshold = threshold, 
                   connectivity = connectivity, 
                   in_place = False)
        
        labels_out = cc3d.connected_components(dusted, 
                                       connectivity = connectivity)
        
        labelled = xr.DataArray(labels_out, 
                        coords = {"y": file_conc.y, "x": file_conc.x}, 
                        dims = ["y", "x"],
                        name = "labels")
        
        # assign ID for basins
        isf_mask_basins = arr_def_ismask['Iceshelf_extrap'].where(isf_only_mask > 0)
        # cut connected areas to area covered by basin stuff
        labelled_isf = labelled.where(np.isfinite(isf_mask_basins))
        
        # creating the mask
        new_mask = isf_mask_basins.copy()
        
        new_mask = new_mask.where(
            new_mask != 58, 57).where(
            new_mask != 151, 99).where(
            new_mask != 109, 107).where(
            new_mask != 116, 5).where(
            new_mask != 143, 97).where(
            new_mask != 137, 99)
        
                    
        arr_def_ismask['isf_name'].loc[{'Nisf': 57}] = 'Ross'
        arr_def_ismask['isf_name'].loc[{'Nisf': 58}] = np.nan
        
        if FRIS_one:
            new_mask = new_mask.where(new_mask != 104, 103)
            arr_def_ismask['isf_name'].loc[{'Nisf': 103}] = 'Filchner-Ronne'
            arr_def_ismask['isf_name'].loc[{'Nisf': 104}] = np.nan

        arr_def_ismask['isf_name'] = arr_def_ismask['isf_name'].dropna('Nisf')
        

    

    
    
    if IMBIE_mask:
        mask_file = new_mask.rename('ISF_mask'), 

    else:
        mask_file = new_mask
    
    return mask_file

In [ ]:
isf_mask_IMBIE = xr.open_dataset(inputpath_data + '')

In [ ]:
print('handling coordinates')

### Create latlon coordinates
meshx, meshy = np.meshgrid(xx,yy)
meshlon,meshlat = uf.change_coord_stereo_to_latlon(meshx,meshy)
file_msk['longitude'] = (['y', 'x'],  meshlon)
file_msk['latitude'] = (['y', 'x'],  meshlat)
lon = file_msk['longitude']
lat = file_msk['latitude']

In [ ]:
file_conc = file_isf_conc
arr_def_ismask = isf_mask_IMBIE

In [ ]:
mask_test = def_isf_mask(isf_mask_IMBIE, file_msk, file_isf_conc, lon, lat, FRIS_one=True, 
                 IMBIE_mask=True, variable_geometry=False, connectivity = 4, threshold = 4)

In [ ]:
isf_mask = file_msk.copy()
# only ice shelves
isf_only_mask = file_conc > 0

# assign ID for basins
isf_mask_basins = arr_def_ismask['Iceshelf_extrap'].where(isf_only_mask > 0)

In [ ]:
isf_mask_basins.plot()

In [ ]:
FRIS_one=True

# creating the mask
new_mask = isf_mask_basins.copy()

arr_def_ismask['isf_name'].loc[{'Nisf': 57}] = 'Ross'
arr_def_ismask['isf_name'].loc[{'Nisf': 58}] = np.nan

if FRIS_one:
    new_mask = new_mask.where(new_mask != 104, 103)
    arr_def_ismask['isf_name'].loc[{'Nisf': 103}] = 'Filchner-Ronne'
    arr_def_ismask['isf_name'].loc[{'Nisf': 104}] = np.nan

arr_def_ismask['isf_name'] = arr_def_ismask['isf_name'].dropna('Nisf')

In [ ]:
#whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
#                                          inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
#                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
#                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True, variable_geometry=False,
#                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
#                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
#                                          write_metadata='yes')

whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          inputpath_data+'Mask_Iceshelf_4km_IMBIE_withNisf.nc', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True, variable_geometry=False,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                          write_metadata='yes')

# Write to netcdf
print('------- WRITE TO NETCDF -----------')
#whole_ds.to_netcdf(outputpath_mask + 'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc','w')


In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_isf_conc = file_mask_orig_cut['isf_conc']
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

xx = file_mask_orig_cut['x']
yy = file_mask_orig_cut['y']

whole_ds_tt = xr.open_dataset(outputpath_mask + 'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc')

nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_var_of_int = whole_ds_tt[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
out_2D.to_netcdf(outputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
out_1D.to_netcdf(outputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
    
# double check here which ice shelves we want    

In [ ]:
plume_param_options = ['cavity','lazero', 'local']

plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

# Compute the ice draft
ice_draft_pos = file_draft
# Be careful with ice shelf 178 and 195 - they have a negative ice draft
# I don't know how to fix it at the moment so I put it to nan
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 178, np.nan)
#ice_draft_pos = ice_draft_pos.where(plume_var_of_int['ISF_mask'] != 195, np.nan)

ice_draft_neg = -1*ice_draft_pos


plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int)
print('------ WRITE TO NETCDF -------')
plume_charac.to_netcdf(outputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

DO THE MASKS FOR 8 KM GRID AS WELL

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask ='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'


file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v3_aggregated8km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
file_msk = file_mask_orig_cut['mask_0_1_2']  #0 = ocean, 1 = ice shelves, 2 = grounded ice

In [ ]:
file_bed_orig = -1*file_mask_orig_cut['bed']
file_draft = (file_mask_orig_cut['thickness'] - file_mask_orig_cut['surface']).where(file_msk==1)
file_isf_conc = file_mask_orig_cut['isf_conc']

xx = file_msk['x']
yy = file_msk['y']

dx = abs(xx[1] - xx[0])
dy = abs(yy[1] - yy[0])

In [ ]:
whole_ds = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=False, variable_geometry=False,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                          write_metadata='yes')

# Write to netcdf
print('------- WRITE TO NETCDF -----------')
whole_ds.to_netcdf(outputpath_mask + 'BedMachine_8km_isf_masks_and_info_and_distance_oneFRIS.nc','w')
